# GraphRAG using Oracle Database 23ai

## GraphRAG using Langchain and Oracle Graph on Oracle Database 23ai

Source: https://medium.com/oracledevs/graphrag-using-langchain-and-oracle-graph-on-oracle-database-23ai-part-1-dc76b48a4ca1

This notebook show simple examples of how we can use **SQL Property Graphs** on **Oracle Database 23ai** with **AI services like ChatGPT** to provide **more context for the LLM** to answer questions.

## Prerequisites

1. Access to a 23ai Oracle Database with a graph enabled user.


### Accessing the Oracle DB 23ai

In [ ]:
import oracledb
import getpass

In [ ]:
# Use python-oracledb in Thick mode (https://python-oracledb.readthedocs.io/en/latest/user_guide/installation.html)

oracledb.init_oracle_client()

In [ ]:
# Connection string details for Autonomous DB

file=open('/home/datascience/config/db_credentials.txt','r')
un=file.readline().strip()
cs=file.readline().strip()

pw = getpass.getpass(f'Enter password for {un}@{cs}: ')

In [ ]:
# Test DB connection

with oracledb.connect(user=un, password=pw, dsn=cs) as connection:
    with connection.cursor() as cursor:
        sql = """select sysdate from dual"""
        for r in cursor.execute(sql):
            print(r)

In [ ]:
# Get DB connection

try:
    connection = oracledb.connect(user=un, password=pw, dsn=cs)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

In [ ]:
# Open a cursor

cursor = connection.cursor()

### Set up tables and data

In [ ]:
# Statements to create tables

create_movie_table = """
    CREATE TABLE IF NOT EXISTS MOVIES (
        MOVIE_ID        NUMBER,
        TITLE           VARCHAR2(400),
        GENRES          JSON,
        SUMMARY         VARCHAR2(16000)
    )
"""

create_customer_table = """
    CREATE TABLE IF NOT EXISTS CUSTOMERS (
        CUST_ID        NUMBER,
        FIRSTNAME      VARCHAR(200),
        LASTNAME       VARCHAR(200)
    )
"""

create_watched_table = """
    CREATE TABLE IF NOT EXISTS WATCHED_MOVIE (
        DAY_ID            TIMESTAMP(6),      
        MOVIE_ID          NUMBER,
        PROMO_CUST_ID     NUMBER
    )
"""

In [ ]:
# Create tables

cursor.execute(create_movie_table)
cursor.execute(create_customer_table)
cursor.execute(create_watched_table)        

In [ ]:
# Statements to clean up tables

truncate_movie_table = """TRUNCATE TABLE MOVIES DROP STORAGE"""
truncate_customer_table = """TRUNCATE TABLE CUSTOMERS DROP STORAGE"""
truncate_watched_table = """TRUNCATE TABLE WATCHED_MOVIE DROP STORAGE"""

In [ ]:
# Truncate tables

cursor.execute(truncate_movie_table)
cursor.execute(truncate_customer_table)
cursor.execute(truncate_watched_table)

In [ ]:
# Populate tables with data

cursor.execute("""
    INSERT INTO MOVIES (MOVIE_ID, TITLE, GENRES, SUMMARY) VALUES
        (1, 'Inception', '{"Action": "Sci-Fi"}', 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.'),
        (2, 'The Matrix', '{"Action": "Sci-Fi"}', 'A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.'),
        (3, 'The Godfather', '{"Drama": "Crime"}', 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'),
        (4, 'Titanic', '{"Romance": "Drama"}', 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'),
        (5, 'Toy Story', '{"Animation": "Adventure"}', 'A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy''s room.')
""")

cursor.execute("""
    INSERT INTO CUSTOMERS (CUST_ID, FIRSTNAME, LASTNAME) VALUES
        (101, 'John', 'Doe'),
        (102, 'Jane', 'Smith'),
        (103, 'Sam', 'Wilson'),
        (104, 'Emily', 'Clark'),
        (105, 'Michael', 'Johnson')
""")

cursor.execute("""
    INSERT INTO WATCHED_MOVIE (DAY_ID, MOVIE_ID, PROMO_CUST_ID) VALUES
        (TO_TIMESTAMP('2024-10-30 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 1, 101),
        (TO_TIMESTAMP('2024-10-31 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 2, 101),
        (TO_TIMESTAMP('2024-09-30 12:34:56.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 3, 101),
        (TO_TIMESTAMP('2024-10-31 09:15:23.654321', 'YYYY-MM-DD HH24:MI:SS.FF'), 2, 102),
        (TO_TIMESTAMP('2024-11-01 16:45:12.987654', 'YYYY-MM-DD HH24:MI:SS.FF'), 3, 103),
        (TO_TIMESTAMP('2024-11-02 18:22:43.123456', 'YYYY-MM-DD HH24:MI:SS.FF'), 4, 104),
        (TO_TIMESTAMP('2024-11-03 19:00:00.000000', 'YYYY-MM-DD HH24:MI:SS.FF'), 1, 105),
        (TO_TIMESTAMP('2024-11-04 20:00:00.000000', 'YYYY-MM-DD HH24:MI:SS.FF'), 2, 105),
        (TO_TIMESTAMP('2024-11-03 21:01:00.000000', 'YYYY-MM-DD HH24:MI:SS.FF'), 1, 102)
""")

cursor.execute("""COMMIT""")

### Define a SQL Property Graph using SQL/PGQ as part of SQL:2023

In [ ]:
# Drop the graph if it exists

sql = """DROP PROPERTY GRAPH IF EXISTS CUSTOMER_WATCHED_MOVIES"""
cursor.execute(sql)

In [ ]:
# Create the graph if it not exists

sql = """
    CREATE PROPERTY GRAPH IF NOT EXISTS CUSTOMER_WATCHED_MOVIES
        VERTEX TABLES (
            CUSTOMERS
                KEY(CUST_ID)
                LABEL Customer
                PROPERTIES ARE ALL COLUMNS,
            MOVIES
                KEY(MOVIE_ID)
                LABEL Movie
                PROPERTIES ARE ALL COLUMNS
        )
        EDGE TABLES(
            WATCHED_MOVIE AS WATCHED
                KEY (DAY_ID, MOVIE_ID, PROMO_CUST_ID)
                SOURCE KEY (PROMO_CUST_ID) REFERENCES CUSTOMERS (CUST_ID)
                DESTINATION KEY (MOVIE_ID) REFERENCES MOVIES (MOVIE_ID)
                LABEL WATCHED
                PROPERTIES ARE ALL COLUMNS
        )
"""
cursor.execute(sql)

### Query the SQL Property Graph using SQL/PGQ

In [ ]:
# Run a first basic query: Customers that watched a movie

sql = """
    SELECT MOVIE_TITLE, LASTNAME, FIRSTNAME
    FROM GRAPH_TABLE(
        CUSTOMER_WATCHED_MOVIES
        MATCH (c IS CUSTOMER) -[w IS WATCHED]-> (m IS MOVIE)
        COLUMNS(c.FIRSTNAME AS FIRSTNAME, c.LASTNAME AS LASTNAME, m.TITLE AS MOVIE_TITLE, w.DAY_ID as DAY_WATCHED)
    )
    ORDER BY MOVIE_TITLE, LASTNAME, FIRSTNAME
"""
for row in cursor.execute(sql):
    print(row)

In [ ]:
# Run a second basic query: Customers that watched the same movie

sql = """
    SELECT DISTINCT
        MOVIE_TITLE,
        C1_NAME, 
        C2_NAME
    FROM GRAPH_TABLE(
        CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER) -[w1 IS WATCHED]-> (m IS MOVIE) <-[w2 IS WATCHED]- (c2 IS CUSTOMER)
        WHERE c1.CUST_ID <> c2.CUST_ID
        COLUMNS(
            c1.CUST_ID AS C1_ID, 
            c2.CUST_ID AS C2_ID, 
            c1.FIRSTNAME || ' ' || c1.LASTNAME AS C1_NAME, 
            c2.FIRSTNAME || ' ' || c2.LASTNAME AS C2_NAME, 
            m.TITLE AS MOVIE_TITLE
        )
    )
    ORDER BY
        MOVIE_TITLE, C1_NAME, C2_NAME
"""
for row in cursor.execute(sql):
    print(row)

### Use SQL Property Graphs and Langchain to provide more context for Generative AI queries

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
file=open('/home/datascience/config/openai_credentials.txt','r')
oai=file.readline().strip()

In [ ]:
# Set up OpenAI LLM

print ("WARNING: The step will fail if the API key is not present or is incorrect.")
print ("Please update the OpenAI_API_key before calling the llm the next step.")

# Set the LLM to get response
llm = ChatOpenAI(
  model_name = 'gpt-4.1-mini',
  temperature = 0.1,
  openai_api_key = oai,
  max_tokens = 2000
)
print("The LLM model you will use is OpenAI ChatGPT 4.0 Mini")

In [ ]:
from langchain_core.prompts import PromptTemplate

**Task**: For every movie that a customer has watched, what is the **genre** based on the movie summary?  
Use a **prompt template** to find the genre.

In [ ]:
# Create a prompt template

template = "What is the genre of {movie} based on this summary: {summary}?"
prompt = PromptTemplate.from_template(template)

# Create a chain with the prompt and LLM
chain = prompt | llm

In [ ]:
# Query the graph: Find all movies that were watched by a customer, and get the movie title and summary back as the result set.

sql = """
    SELECT DISTINCT 
        MOVIE_TITLE, 
        MOVIE_SUMMARY
    FROM GRAPH_TABLE( 
        CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        COLUMNS (m.title as MOVIE_TITLE, m.summary as MOVIE_SUMMARY) 
    )
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the executed query
rows = cursor.fetchall()
print(rows)

In [ ]:
# Invoke the chain based on the response and print the results including the generated genre

for row in rows:
    # Run the chain and print the output
    result = chain.invoke({'movie': row[0], 'summary': row[1]})
    print(result.content)

**Task**: Based on the movies a customer has watched, containing movie titles, genres and summaries. How would you describe the movie watching preferences of this user?

In [ ]:
# Query the graph: Find all movies, customer 101 has watched

sql = """
    SELECT DISTINCT
        MOVIE_TITLE, 
        MOVIE_SUMMARY
    FROM GRAPH_TABLE( 
        CUSTOMER_WATCHED_MOVIES
        MATCH (c IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        WHERE c.CUST_ID = 101
        COLUMNS (m.title as MOVIE_TITLE, m.summary as MOVIE_SUMMARY) 
    )
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the executed query
rows = cursor.fetchall()
print(rows)

In [ ]:
# Create a new prompt template

template = "Based on the movies a customer has watched, containing movie titles, genres and summaries. How would you describe the movie watching preferences of this user? {data}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [ ]:
# Invoke the chain and print the results

result = chain.invoke({'data': rows})
print(result.content)

The analysis is about the customer habits of watching movies. This can be useful for making informed decisions on what types of movies customers tend to like, so we can provide tailored recommendations to them.

**Task**: Instead of writing a SQL query to ask which user has watched the most movies, ask the LLM?

In [ ]:
# Query the graph: Find all movies watched by any customer

sql = """
    SELECT *
    FROM GRAPH_TABLE( 
        CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        COLUMNS (c1.CUST_ID as customer_id, m.MOVIE_ID as movie_id, e1.DAY_ID as date_watched) 
    )
    ORDER BY
        customer_id,
        movie_id,
        date_watched        
"""

# Execute the SQL query
cursor.execute(sql)

# Fetch all rows from the result set
result = cursor.fetchall()

# Print the rows
for row in result:
  print(row)

In [ ]:
# Create another prompt template

template = "Based on the movies that customers have watched, containing customer IDs, movie IDs and date watched, Which customer has watched the most movies? {data}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [ ]:
# Invoke the chain and print the results

result = chain.invoke({'data': rows})
print(result.content)

In [ ]:
cursor.execute("""
    INSERT INTO WATCHED_MOVIE (DAY_ID, MOVIE_ID, PROMO_CUST_ID) VALUES
        (TO_TIMESTAMP('2025-06-05 12:00:00.000000', 'YYYY-MM-DD HH24:MI:SS.FF'), 5, 101)
""")

cursor.execute("""COMMIT""")

In [ ]:
# Re-run the query

sql = """
    SELECT *
    FROM GRAPH_TABLE( 
        CUSTOMER_WATCHED_MOVIES
        MATCH (c1 IS CUSTOMER)-[e1 IS WATCHED]->(m IS MOVIE)
        COLUMNS (c1.CUST_ID as customer_id, m.MOVIE_ID as movie_id, e1.DAY_ID as date_watched) 
    )
"""
cursor.execute(sql)
rows = cursor.fetchall()

In [ ]:
# Re-invoke the chain and print the results

result = chain.invoke({'data': rows})
print(result.content)